# Data 

In [2]:
import pandas as pd
import numpy as np

In [4]:
fairprice_data = pd.read_csv(r"C:\Users\User\Desktop\Fresh\YoRipe\Fairprice\fairprice_output.csv")

In [5]:
fairprice_data

,Unnamed: 0,urls,recipename,servings,preptime,ingredients,instructions,level_of_difficulty,image_url,video_url
0,0,https://www.fairprice.com.sg/recipes/fragrant-...,Fragrant Bitter Gourd Chicken,8 pax,20 mins,1 Pasar Jumbo Fresh Chicken*\n2 pcs Lotus Leav...,Remove spare parts from chicken cavity. Rinse ...,Easy,NaN,https://www.youtube.com/embed/kEu3FM0fkpo
1,1,https://www.fairprice.com.sg/recipes/treasured...,Treasured Sesame Oil Duck,8 pax,15 mins,20ml Knife 100% Pure Sesame Oil\n½ Duck (cut i...,Deep fry the yam sticks till crispy. Set aside...,Intermediate,NaN,https://www.youtube.com/embed/5vW4rrPwPj0
2,2,https://www.fairprice.com.sg/recipes/fruitful-...,Fruitful Sweet Prawns,4 pax,20 mins,10 pcs Diamond Live Frozen White Prawns*\n40g ...,Add oil and fry unshelled thawed prawns over h...,Easy,NaN,https://www.youtube.com/embed/vD64-6io6oQ
3,3,https://www.fairprice.com.sg/recipes/pacific-c...,Pacific Clam with Roasted Sesame Dressing,10,15 mins,"1 can Pacific clams, drained\n20 Red cherry to...",Method\nCut a small cross lightly at the botto...,Easy,https://www.fairprice.com.sg/wp-content/upload...,NaN
4,4,https://www.fairprice.com.sg/recipes/new-zeala...,New Zealand Superior Wild Abalone with Red Chi...,6 pax,15 mins,1 can Golden Chef New Zealand Superior Wild Ab...,Method\nBoil Golden Chef New Zealand Superior ...,Easy,https://www.fairprice.com.sg/wp-content/upload...,NaN
...,...,...,...,...,...,...,...,...,...,...
111,111,https://www.fairprice.com.sg/recipes/chicken-b...,Chicken Brown Rice,10 pax,30mins,For Chicken\n1.8kg Whole Chicken\n6L Water\n20...,Method\nChicken\nHeat up water in a deep pot u...,Intermediate,NaN,https://www.youtube.com/embed/424U2yViB7Y
112,112,https://www.fairprice.com.sg/recipes/healthy-f...,Healthy Fried Brown Rice,2 - 4 pax,30mins,2 tbsp Virgin Coconut Oil\n1 knob of Fresh Gin...,"Method\nOn medium fire, heat oil in a large sa...",Easy,NaN,https://www.youtube.com/embed/FOaV8RAEzxM
113,113,https://www.fairprice.com.sg/recipes/pan-fried...,Pan Fried Fukuyama Scallops with Citrus Tofu S...,8 pax,30mins,"300g Fukuyama Scallops, thawed and rinsed\n2pc...",Method\nPat the scallops dry with a clean kitc...,Easy,https://www.fairprice.com.sg/wp-content/upload...,NaN
114,114,https://www.fairprice.com.sg/recipes/fish-maw-...,Fish Maw Soup in Young Coconut,4 pax,30mins,750ml Swanson Chicken Broth\n4 Young Coconuts\...,"Method\nTo prepare the meat balls, mix the ing...",Easy,https://donate.fairprice.com.sg/wp-content/upl...,NaN


In [ ]:
fairprice_data.rename(columns={"Unnamed: 0": "recipeid"}, inplace=True)

In [ ]:
fairprice_data['recipeid'] = range(202022, len(fairprice_data) + 202022) 

In [ ]:
fairprice_data.columns

In [ ]:
def edit_string1(s):
    s.replace("â€™", ",")
    return s 

In [ ]:
coding: utf-8 

def edit_string2(s):
    s.replace("Â½", "1/2")
    return s 

In [ ]:
edit_string2("Â½ cup Brown sugar")

In [ ]:
fairprice_data['recipename'] = fairprice_data['recipename'].astype(str).apply(edit_string1)

In [ ]:
fairprice_data_list = fairprice_data['recipename'].str.lower()

In [ ]:
def first_digit(s):
    for i, c in enumerate(str(s)):
        if c.isdigit():
            return c
            break

In [ ]:
fairprice_data['servings'] = fairprice_data['servings'].apply(first_digit)

In [ ]:
import re
def cooking_split(line):
    lines = re.split(r';|,', line)
    return lines

In [ ]:
fairprice_data[['preptime', 'cookingtime']] = fairprice_data['preptime'].str.split(";", expand = True) 

In [ ]:
fairprice_data['cookingtime'] = fairprice_data['cookingtime'].fillna("NA")

In [ ]:
from collections import defaultdict
import re

def humantime2minutes(s):
    d = {
      'w':      7*24*60,
      'week':   7*24*60,
      'weeks':  7*24*60,
      'd':      24*60,
      'day':    24*60,
      'days':   24*60,
      'h':      60,
      'hr':     60,
      'hour':   60,
      'hours':  60,
    }
    mult_items = defaultdict(lambda: 1).copy()
    mult_items.update(d)

    if s != "NA":
        parts = re.search(r'^(\d+)([^\d]*)', str(s).lower().replace(' ', ''))
        if parts:
            return int(parts.group(1)) * mult_items[parts.group(2)]
        else:
            return 0

In [ ]:
fairprice_data['preptime'] = fairprice_data['preptime'].apply(humantime2minutes)

In [ ]:
fairprice_data['cookingtime'] = fairprice_data['cookingtime'].apply(humantime2minutes)

In [ ]:
def faster(df):
    s = df["ingredients"].str.split('\n', expand=True).stack()
    i = s.index.get_level_values(0)
    df2 = df.loc[i].copy()
    df2["ingredients"] = s.values
    return df2

In [ ]:
fairprice_data_exploded = faster(fairprice_data)

In [ ]:
fairprice_data_list_ingred = fairprice_data_exploded['ingredients'].str.lower()

# Retrain New Model 

In [ ]:
import gensim

model3 = gensim.models.KeyedVectors.load_word2vec_format('../YoRipe/Dictionaries/GoogleNews-vectors-negative300.bin',
                                                        binary=True)

In [ ]:
words = model3.index2word

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank

In [ ]:
dictionary = []
for i in WORDS:
    dictionary.append([i])

len(dictionary)

In [ ]:
model3_train_input1 = pd.read_excel(r'C:\Users\Solomonzhs\Desktop\Fresh\YoRipe\Re__Data_files_for_your_perusal\ingred_train.xlsx')
model3_train_input2 = pd.read_excel(r'C:\Users\Solomonzhs\Desktop\Fresh\YoRipe\Re__Data_files_for_your_perusal\ingred_changed with spelling correction.xlsx', sheet_name='Sheet2')
model3_train_input3 = pd.read_excel(r'C:\Users\Solomonzhs\Desktop\Fresh\YoRipe\Re__Data_files_for_your_perusal\Dictionary_Addition_2.xlsx', header = None)

In [ ]:
def orig(df):
    s = df["variations"].str.split(',').apply(pd.Series,1).stack()
    s.index = s.index.droplevel(-1)
    s.name = "Variations"
    del df["variations"]
    return df.join(s)

In [ ]:
model3_train_input2 = orig(model3_train_input2)

In [ ]:
sentences1 = model3_train_input1['Sentences'].tolist()
sentences2 = model3_train_input2['Correct spelling'].tolist()
sentences3 = model3_train_input2['Variations'].tolist()
sentences4 = model3_train_input3[0].tolist()

In [ ]:
for i in sentences1: 
    dictionary.append([str(i)])

for j in sentences2: 
    dictionary.append([str(j)])

for k in sentences3: 
    dictionary.append([str(k)])
    
for m in sentences4:
    dictionary.append([str(m)])

In [ ]:
model_latest = gensim.models.Word2Vec(dictionary, min_count=1, workers=8)

In [ ]:
model_latest.save('model_latest.bin')

# Use Model 

In [ ]:
import gensim
new_model = gensim.models.Word2Vec.load('model_latest.bin')

In [ ]:
#words = model.index2word

w_rank = {}
for i,word in enumerate(new_model.wv.vocab):
    w_rank[word] = i
    
WORDS = w_rank
WORDS

len(WORDS)

In [ ]:
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
correction_phrases= []
original_phrases = []
for i in fairprice_data_list_ingred:
    correction_phrases.append(correction(str(i)))
    original_phrases.append(i)

In [ ]:
import math

counterA = Counter(original_phrases)
counterB = Counter(correction_phrases)


def counter_cosine_similarity(c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)

print(counter_cosine_similarity(counterA, counterB) * 100)

In [ ]:
recipe_tuples = list(zip(correction_phrases, original_phrases))
recipe_changed = pd.DataFrame(recipe_tuples, columns = ['Corrected','Original'])

In [ ]:
list1 = list(set(correction_phrases) - set(original_phrases))
len(list1)

In [ ]:
list2 = set(set(original_phrases) - set(correction_phrases))
len(list2)

In [ ]:
recipe_changed['Changed'] = recipe_changed['Original'].apply(lambda  x: 1 if x in list2 else 0)

In [ ]:
recipe_changed.to_excel(r'recipe_changed_new_4.xlsx')

# Upper Case 

In [ ]:
fairprice_data_v2_load = pd.read_excel(r'C:\Users\Solomonzhs\Desktop\Fresh\YoRipe\Fairprice\recipe_changed_new_4.xlsx')

In [ ]:
def index_upper(string):
    index = []
    for i in range(len(string)):
        if string[i].isupper():
            print(string[i])
            index.append(i)
    return index

In [ ]:
fairprice_data_exploded['index_upper_list'] = fairprice_data_exploded['ingredients'].astype(str).apply(index_upper)

In [ ]:
fairprice_data_exploded

In [ ]:
fairprice_data_exploded = fairprice_data_exploded.reset_index() 

In [ ]:
fairprice_data_exploded['ingredients2'] = fairprice_data_v2_load['Corrected']

In [ ]:
def upper_index(s, indexes):
    s = "".join(c.upper() if i in indexes else c for i, c in enumerate(s))
    return s

In [ ]:
fairprice_data_exploded['ingredients3'] = fairprice_data_exploded.apply(lambda row: upper_index(row['ingredients2'], 
                                                                                                row['index_upper_list']), axis=1)

In [ ]:
fairprice_data_exploded

In [ ]:
def up_case(x):
    return x.title()

fairprice_data_v2['ingredients'] = fairprice_data_v2_load['Corrected']#.apply(up_case)

In [ ]:
string1 = 'geeksforgeeks'
string2 = 'GeeksforGeeks'
string3 = ''
for i in range(len(string2)):
    if not string2[i].isupper():
        string3 += string1[i]
    else:`
        string3 += string1[i].upper()
string3

In [ ]:
outcome = []

for i in range(len(fairprice_data_v2_load)):
    print("1-" + fairprice_data_v2_load['Corrected'][i])
    print("3-" + fairprice_data_exploded['ingredients'][i])
    string3 = ''
    for j in fairprice_data_exploded['ingredients'][i]:
        #print(j)
        if not j.isupper():
            string3 += j
        else:
            string3 = j.upper()
    print("2-" + string3)
#     for j in fairprice_data_v2['ingredients'][i]:
#         print(j)
#         string3 = ''
#         for k in fairprice_data['ingredients'][i]:
#             if not k.isupper():
#                 string3 += k
#             else:
#                 string3 += k.upper()
#         j = string3
#         print(j)

In [ ]:
fairprice_data_v2